#### Download and extract VCTK dataset from Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
download_file_from_google_drive('1-1mwn38ifDNO7B_950MsdnM3yyAEYczP', 'VCTK.zip')

In [0]:
download_file_from_google_drive('10C6xt5wCuIgNp2GxULJISsDGRPZFKCMp', 'model.hdf5')

In [0]:
mkdir '/content/VCTK/'

In [0]:
!cd '/content/VCTK/' && unzip -q /content/VCTK.zip

#### Install required dependencies and import

In [0]:
!pip install webrtcvad
!pip install pydub
import webrtcvad
import struct
import pandas as pd, glob

#### Download code and make directory for preprocessed files

In [0]:
!git clone https://github.com/sagar-spkt/SV2MTTS.git

In [0]:
%cd SV2MTTS/

In [0]:
!git checkout VCTK

In [0]:
!mkdir /content/VCTK-Num-Embbded

### import Preprocessing function

In [0]:
from preprocess import wav_to_speaker_embeddings

In [0]:
wav_to_speaker_embeddings('/content/VCTK', '/content/VCTK-Num-Embbded', 'VCTK', '/content/model.hdf5')

W0722 11:34:49.618400 140329872258944 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
  1%|          | 7/1383 [00:16<54:58,  2.40s/it]

### File Saving to Drive

In [0]:
import time
import os

start = time.time()

os.system("tar -cvf VCTK-Num-32-Embbded-16-siltrim-23.tar /content/VCTK-Num-Embbded")

end = time.time()
print("Elapsed time: %s"%(end - start,))

In [0]:
!rsync -P '/content/SV2MTTS/VCTK-Num-32-Embbded-16-siltrim-23.tar' '/content/drive/My Drive/ML/dataset/LibriTTS/'